<a href="https://colab.research.google.com/github/ayushjain1144/NER/blob/master/NER_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook



/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


# Loading the features and vocabulary

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

!ln -s content/gdrive/My\ Drive/NER /ner_dir

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
%cd ..
%cd /ner_dir


/
/content/gdrive/My Drive/NER


In [4]:
!ls

activations.py			      params-50.npy
cal_statistics.py		      params-50-random_initialization.npy
Conll.ipynb			      params-50-range_initialization.npy
dataset				      params-550-range_initialization.npy
fig1-loss-Exp1.jpg		      params-600-range_initialization.npy
fig1-loss-Exp2.jpg		      params-650-range_initialization.npy
fig1-loss-Exp3.jpg		      params-700-range_initialization.npy
fig1-loss-Exp4.jpg		      params-750-range_initialization.npy
fig1-loss-Exp5.jpg		      params-800-range_initialization.npy
initial_experiment.ipynb	      params-850-range_initialization.npy
initialization.py		      params-900-range_initialization.npy
loss.py				      params-950-range_initialization.npy
NER_NN.ipynb			      __pycache__
NER_NN_network.ipynb		      README.md
nn.py				      results_bal.txt
params-0.npy			      results.txt
params-0-random_initialization.npy    statistics.py
params-0-range_initialization.npy     test_features.npy
params-100.npy			      train_features.npy
params-100-random_

In [5]:
import activations
import loss
import initialization as init_layer

In [6]:
vocab = np.load('vocab.npy')
train_features = np.load('train_features.npy').T
test_features = np.load('test_features.npy').T
val_features = np.load('val_features.npy').T
y_train = np.load('ytrain.npy').T
y_val = np.load('yval.npy').T
y_test = np.load('ytest.npy').T

In [7]:
print(train_features.shape)
print(vocab.shape)
print(test_features.shape)
print(val_features.shape)
print(y_train.shape)
print(y_test.shape)
print(y_val.shape)

(900, 204566)
(26872, 300)
(900, 46665)
(900, 51577)
(10, 204566)
(10, 46665)
(10, 51577)


In [8]:
train_features[:10]

array([[ 1.        , -0.92605459,  0.00982666, ...,  1.38121068,
         0.05078125,  1.46252757],
       [ 0.        , -1.13792351,  0.2265625 , ...,  0.8632994 ,
        -0.09326172, -0.07399186],
       [ 0.        , -0.7880129 ,  0.28125   , ...,  0.76144395,
         0.06494141,  0.03143081],
       ...,
       [ 0.        , -0.43820235, -0.03540039, ...,  0.7176954 ,
        -0.08154297,  0.11263644],
       [ 0.        , -0.95179252,  0.14746094, ...,  0.69231712,
         0.13085938,  0.80112245],
       [ 0.        , -1.45894089,  0.12890625, ..., -1.61798501,
         0.12597656, -0.11506672]])

In [9]:
y_train[:10]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 1., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [10]:
C = 1
word_vector_dim = 300
num_tags = 10

# Neural Netwwork Architecture

In [11]:
nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  #{"layer_size": 300, "activation": "relu"},
  #{"layer_size": 100, "activation": "relu"},
  {"layer_size": 10, "activation": "sigmoid"}
]

In [12]:
nn_architecture[1]['activation']

'sigmoid'

# Initialize Parameters

In [13]:
def initialize_parameters(nn_architecture, initialization = "range_initialization", seed=5):

  parameters = {}
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):

    if initialization == "range_initialization":
      parameters['W' + str(i)] = init_layer.range_initializtion(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)
      # print(parameters['W1'])
    else:
      parameters['W' + str(i)] = init_layer.random_initialization(nn_architecture[i]["layer_size"],
                                            nn_architecture[i - 1]["layer_size"], seed)

    parameters['b' + str(i)] = np.zeros((nn_architecture[i]["layer_size"], 1))

  return parameters

In [14]:
param = initialize_parameters(nn_architecture, initialization="range_initialization")
for k in param.keys():
  print(f"{k}: {param[k].shape}")

W1: (10, 900)
b1: (10, 1)


# Forward Propogation

In [15]:
# Z = W * X + b
# Here A is output of previous layer

def linear_forward(A_prev, W, b):
  return np.dot(W, A_prev) + b

# apply activation h:  A = h(X) 
def apply_activation(A, activation, alpha=0.01):

  if activation == "sigmoid":
    return activations.sigmoid(A)
  elif activation == "tanh":
    return activations.tanh(A)
  elif activation == "relu":
    return activations.relu(A)
  elif activation == "leaky_relu":
    return activations.leaky_relu(A, alpha)
  elif activation == "softmax":
    return loss.softmax(A)
  elif activation == 'none':
    #print("None activation used")
    return A
  else:
    print(f"ERROR: {activation} activation not supported")
    sys.exit(1)
  
# driver forward propogation
def model_forward(X, parameters, nn_architecture, alpha=0.01):

  forward_cache = []
  A = X
  num_layers = len(nn_architecture)

  for i in range(1, num_layers):
    A_prev = A
    # print(f"{A_prev.shape}: for")
    W = parameters["W" + str(i)]
    b = parameters["b" + str(i)]

    Z = linear_forward(A_prev, W, b)
    activation = nn_architecture[i]['activation']
    # print(activation)
    A = apply_activation(Z, activation, alpha)
    # print(A)
    forward_cache.append(((A_prev, W, b), Z))

  # print("after loop")
  # print(A)
  return A, forward_cache  

# Backpropogation 

In [16]:
def linear_backward(dz, cache):
  A_prev, W, b = cache
  m = A_prev.shape[1]
  # print(m)
  dw = (1 / m) * np.dot(dz, A_prev.T)
  db = (1 / m) * np.sum(dz, axis=1, keepdims=True)
  dA_prev = np.dot(W.T, dz)

  assert dA_prev.shape == A_prev.shape
  assert dw.shape == W.shape
  assert db.shape == b.shape

  return dA_prev, dw, db      

In [17]:
def apply_activation_backward(dA, cache, activation_fn):
  linear_cache, activation_cache = cache

  if activation_fn == "sigmoid":
    dZ = activations.sigmoid_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "tanh":
    dZ = activations.tanh_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "relu":
    dZ = activations.relu_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)

  elif activation_fn == "leaky_relu":
    dZ = activations.leaky_relu_backward(dA, activation_cache)
    dA_prev, dw, db = linear_backward(dZ, linear_cache)
  
  elif activation_fn == "none":
    dZ = dA
    dA_prev, dw, db = linear_backward(dZ, linear_cache)
  
  else:
    print("Activation not available")
    sys.exit(1)

  return dA_prev, dw, db

  


In [18]:
def model_backward(AL, y, caches, nn_architecture):
  
  y = y.reshape(AL.shape)
  L = len(caches)
  # print(f"Len of cache: {len(caches)}")
  grads = {}

  dAL = np.divide(AL - y, np.multiply(AL, 1 - AL))
  # print(caches.keys())
  # print(grads.keys())
  # print(len(caches))
  grads["dA" + str(L - 1)], grads["dW" + str(L)], grads["db" + str(L)] = apply_activation_backward(dAL, caches[L-1], nn_architecture[L]['activation'])

  for l in range(L - 1, 0, -1):
    # print(l)
    current_cache = caches[l - 1]
    grads["dA" + str(l - 1)], grads["dW" + str(l)],  \
        grads["db" + str(l)] = apply_activation_backward(
            grads["dA" + str(l)], current_cache, 
            nn_architecture[l]['activation']
        )
    
  return grads

In [19]:
def update_parameters(parameters, grads, lr):

  L = len(parameters) // 2

  for l in range(1, L + 1):
    parameters["W" + str(l)] = parameters["W" + str(l)] - \
            lr * grads["dW" + str(l)]
    parameters["b" + str(l)] = parameters["b" + str(l)] - \
            lr * grads["db" + str(l)]
  return parameters

In [20]:
def model(X, y, nn_architecture, exp_name, initialization='range_initialisation', lr=0.01, num_iterations=2000, print_cost=True, checkpoint_initialisation=None):

  np.random.seed(1)

  if checkpoint_initialisation == None:
    parameters = initialize_parameters(nn_architecture, initialization)
  else:
    print(f"Loading checkpoints from file {checkpoint_initialisation}")
    parameters = np.load(checkpoint_initialisation, allow_pickle=True).item()

  cost_list = []
  val_loss = []
  val_list = []
  train_list = []
  #iterate over iterations

  for i in tqdm_notebook(range(0, num_iterations)):

    #forward step
    AL, caches = model_forward(X, parameters, nn_architecture)
    AL_val, _ = model_forward(val_features, parameters, nn_architecture)
    cost = loss.cross_entropy_loss(AL, y)
    val_cost = loss.cross_entropy_loss(AL_val, y_val)
    # print(cost)
    grads = model_backward(AL, y, caches, nn_architecture)

    parameters = update_parameters(parameters, grads, lr)

    
    print(f"The cost after {i + 1} iterations is: Train: {cost: .4f}, Val: {val_cost: .4f}")

    train_acc = accuracy(X, parameters, y, nn_architecture)
    acc = accuracy(val_features, parameters, y_val, nn_architecture)
    print(f"After {i + 1} iterations: Val Acc: {acc}; Train Acc: {train_acc}")
    val_list.append(acc)
    train_list.append(train_acc)

    
    cost_list.append(cost)
    val_loss.append(val_cost)
    # if i % 50 == 0:
    #   print(f"Creating checkpoint for {i}th iteration")
    #   np.save(f'params-{i}-{initialization}.npy', parameters)

    #plotting cost curve

  plt.figure(figsize=(10, 6))
  plt.plot(cost_list, 'b', label='train_loss')
  plt.plot(val_loss, 'r', label='val_loss')
  plt.legend()
  plt.xlabel("Iterations")
  plt.ylabel("Loss")
  plt.title(f"Loss curve for the learning rate = {lr}")
  plt.savefig(f'fig1-loss-{exp_name}.jpg', bbox_inches='tight')
  plt.figure(figsize=(10, 6))
  plt.plot(val_list, 'g', label="val acc")
  plt.ylim(0, 100)
  plt.legend()
  plt.savefig(f'val-acc-{exp_name}.jpg', bbox_inches='tight')
  plt.figure(figsize=(10,6))
  plt.plot(train_list, 'r', label ="train acc")
  plt.ylim(0, 100)
  plt.legend()
  plt.savefig(f'train-loss-{exp_name}.jpg', bbox_inches='tight')
  plt.xlabel("Iterations")
  plt.ylabel("Accuracy")
  plt.title(f"Accuracy curve for the learning rate = {lr}")

  return parameters

In [21]:
def accuracy(X, parameters, y, nn_architecture):
  probs, caches = model_forward(X, parameters, nn_architecture)
  labels = np.argmax(probs, axis=0)
  gt = np.argmax(y, axis=0)
  accuracy = np.mean(labels==gt) * 100
  
  return accuracy

In [22]:
# Todo

#implement margin loss
#implement regularisation

# Gradient Checking

In [23]:
from numpy.linalg import norm
import imblearn
from imblearn.over_sampling import SMOTE

def dictionary_to_vector(params_dict):
  count = 0

  for key in params_dict.keys():
    new_vector = np.reshape(params_dict[key], (-1, 1))

    if count == 0:
      theta_vector = new_vector
    else:
      theta_vector = np.concatenate((theta_vector, new_vector))
    count += 1

  return theta_vector



In [24]:
def vector_to_dictionary(vector, nn_architecture):

  L = len(nn_architecture)
  parameters = {}
  k = 0

  for l in range(1, L):

    w_dim = nn_architecture[l]['layer_size'] * nn_architecture[l - 1]['layer_size']
    b_dim = nn_architecture[l]['layer_size']

    temp_dim = k + w_dim

    parameters["W" + str(l)] = vector[k:temp_dim].reshape(nn_architecture[l]['layer_size'], nn_architecture[l - 1]['layer_size'])
    parameters["b" + str(l)] = vector[temp_dim:temp_dim + b_dim].reshape(b_dim, 1)

    k += w_dim + b_dim

  return parameters

In [25]:
def gradients_to_vector(gradients):

  valid_grads = [key for key in gradients.keys() if not key.startswith("dA")]
  L = len(valid_grads) // 2

  count = 0

  for l in range(1, L + 1):
    if count == 0:
      new_grads = gradients["dW" + str(l)].reshape(-1, 1)
      new_grads = np.concatenate((new_grads, gradients["db" + str(l)].reshape(-1, 1)))

    else:
      new_grads = np.concatenate((new_grads, gradients["dW" + str(l)].reshape(-1, 1)))
      new_grads = np.concatenate((new_grads, gradients["db" + str(l)].reshape(-1, 1)))

    count += 1
  return new_grads

In [26]:
def forward_prop_cost(X, parameters, Y, nn_architecture):

  AL, _ = model_forward(X, parameters, nn_architecture)
  cost = loss.cross_entropy_loss(AL, Y)
  # print(cost)
  return cost

def gradient_check(parameters, gradients, X, Y, nn_architecture, epsilon=1e-4):

  parameters_vector = dictionary_to_vector(parameters)
  gradients_vector = gradients_to_vector(gradients)

  grads_approx = np.zeros_like(parameters_vector)

  for i in range(len(parameters_vector)):

    theta_plus = np.copy(parameters_vector)
    theta_plus[i] = theta_plus[i] + epsilon
    j_plus = forward_prop_cost(X, vector_to_dictionary(theta_plus, nn_architecture), Y, nn_architecture)


    theta_minus = np.copy(parameters_vector)
    theta_minus[i] = theta_minus[i] - epsilon
    j_minus = forward_prop_cost(X, vector_to_dictionary(theta_minus, nn_architecture), Y, nn_architecture)

    grads_approx[i] = (j_plus - j_minus) / (2 * epsilon)

    #print(f"grads_approx{i}: {grads_approx[i]} and {gradients_vector[i]}")

  

  numerator = norm(gradients_vector - grads_approx)
  denominator = norm(grads_approx) + norm(gradients_vector)

  difference = numerator / denominator

  if difference > 10e-4:
    print(f"Backprop Wrong: difference = {difference}")
  else:
    print(f"Backprop Correct: difference = {difference}")

  return difference





### Execute the following cell for gradient checking. 

In [28]:
# performing graident checking
nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "relu"},
  {"layer_size": 100, "activation": "relu"},
  {"layer_size": 10, "activation": "sigmoid"}
]
def perform_gradient_check():
  parameters = initialize_parameters(nn_architecture)

  perms = np.random.permutation(train_features.shape[1])
  index = perms[:1]
  print(train_features[:, index].shape)
  print(y_train[:, index].shape)

  AL, caches = model_forward(train_features[:, index], parameters, nn_architecture)
  grads = model_backward(AL, y_train[:, index], caches, nn_architecture)
  #print(grads)
  difference = gradient_check(parameters, grads, train_features[:, index], y_train[:, index], nn_architecture)

perform_gradient_check()

(900, 1)
(10, 1)
Backprop Correct: difference = 8.085386956443845e-11


### Execute the following cell for training the model

In [ ]:
# # Experiment 1: Single Layer Perceptron. range initialisatioin, 2000 iterations, relu
# fp = open('results.txt', 'a')

# params = model(train_features, y_train, nn_architecture, exp_name='Exp1', initialization='range_initialization', lr=0.1, num_iterations=1000)


# accuracy(test_features, params, y_test, nn_architecture)

# AL, _ = model_forward(test_features, params, nn_architecture)

# from statistics import print_statistics

# stats = print_statistics(AL, y_test)

# fp.write("Experiment: Single Layer Perceptron. range initialisatioin, 1000 iterations, relu\n\n")
# fp.write(stats)
# fp.close()


# # Experiment 2: Single Layer Perceptron. random initialisatioin, 2000 iterations, relu
# fp = open('results.txt', 'a')

# params = model(train_features, y_train, nn_architecture, exp_name='Exp2', initialization='random_initialization', lr=0.1, num_iterations=1000)


# accuracy(test_features, params, y_test, nn_architecture)

# AL, _ = model_forward(test_features, params, nn_architecture)

# from statistics import print_statistics

# stats = print_statistics(AL, y_test)

# fp.write("Experiment: Single Layer Perceptron. random initialisatioin, 1000 iterations, relu\n\n")
# fp.write(stats)
# fp.close()




# # Experiment 3: Multi Layer Perceptron. range initialisatioin, 1000 iterations, relu
# fp = open('results.txt', 'a')

# nn_architecture = [
#   {"layer_size": 900, "activation": "none"},
#   {"layer_size": 300, "activation": "relu"},
#   {"layer_size": 100, "activation": "relu"},
#   {"layer_size": 10, "activation": "sigmoid"}
# ]

# params = model(train_features, y_train, nn_architecture, exp_name='Exp3', initialization='range_initialization', lr=0.1, num_iterations=300)


# accuracy(test_features, params, y_test, nn_architecture)

# AL, _ = model_forward(test_features, params, nn_architecture)

# from statistics import print_statistics

# stats = print_statistics(AL, y_test)

# fp.write("Experiment: Multi Layer Perceptron. range initialisatioin, 1000 iterations, relu\n\n")
# fp.write(stats)
# fp.close()


# # Experiment 4: Multi Layer Perceptron. range initialisatioin, 1000 iterations, tanh
# fp = open('results.txt', 'a')

# nn_architecture = [
#   {"layer_size": 900, "activation": "none"},
#   {"layer_size": 300, "activation": "tanh"},
#   {"layer_size": 100, "activation": "tanh"},
#   {"layer_size": 10, "activation": "sigmoid"}
# ]

# params = model(train_features, y_train, nn_architecture, exp_name='Exp4', initialization='range_initialization', lr=0.1, num_iterations=300)


# accuracy(test_features, params, y_test, nn_architecture)

# AL, _ = model_forward(test_features, params, nn_architecture)

# from statistics import print_statistics

# stats = print_statistics(AL, y_test)

# fp.write("Experiment: Multiple Layer Perceptron. range initialisatioin, 1000 iterations, tanh\n\n")
# fp.write(stats)
# fp.close()

# # Experiment 5: Multi Layer Perceptron. range initialisatioin, 1000 iterations, sigmoid
# fp = open('results.txt', 'a')

# nn_architecture = [
#   {"layer_size": 900, "activation": "none"},
#   {"layer_size": 300, "activation": "sigmoid"},
#   {"layer_size": 100, "activation": "sigmoid"},
#   {"layer_size": 10, "activation": "sigmoid"}
# ]



# params = model(train_features, y_train, nn_architecture, exp_name='Exp5', initialization='range_initialization', lr=0.1, num_iterations=300)


# accuracy(test_features, params, y_test, nn_architecture)

# AL, _ = model_forward(test_features, params, nn_architecture)

# from statistics import print_statistics

# stats = print_statistics(AL, y_test)

# fp.write("Experiment: Multiple Layer Perceptron. range initialisatioin, 1000 iterations, sigmoid\n\n")
# fp.write(stats)
# fp.close()


# Experiment 6: Multi Layer Perceptron. random initialisatioin, 1000 iterations, relu
fp = open('results.txt', 'a')

nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "relu"},
  {"layer_size": 100, "activation": "relu"},
  {"layer_size": 10, "activation": "sigmoid"}
]

params = model(train_features, y_train, nn_architecture, exp_name='Exp6', initialization='random_initialization', lr=0.1, num_iterations=300)


accuracy(test_features, params, y_test, nn_architecture)

AL, _ = model_forward(test_features, params, nn_architecture)

from statistics import print_statistics

stats = print_statistics(AL, y_test)

fp.write("Experiment: Multiple Layer Perceptron. random initialisatioin, 1000 iterations, relu\n\n")
fp.write(stats)
fp.close()

# Experiment 7: Multi Layer Perceptron. random initialisatioin, 1000 iterations, leaky relu
fp = open('results.txt', 'a')

nn_architecture = [
  {"layer_size": 900, "activation": "none"},
  {"layer_size": 300, "activation": "leaky_relu"},
  {"layer_size": 100, "activation": "leaky_relu"},
  {"layer_size": 10, "activation": "sigmoid"}
]

params = model(train_features, y_train, nn_architecture, exp_name='Exp7', initialization='range_initialization', lr=0.1, num_iterations=300)


accuracy(test_features, params, y_test, nn_architecture)

AL, _ = model_forward(test_features, params, nn_architecture)

from statistics import print_statistics

stats = print_statistics(AL, y_test)

fp.write("Experiment: Multiple Layer Perceptron. random initialisatioin, 1000 iterations, relu\n\n")
fp.write(stats)
fp.close()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


The cost after 1 iterations is: Train:  6.9311, Val:  6.9311
After 1 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 2 iterations is: Train:  6.7110, Val:  6.7112
After 2 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 3 iterations is: Train:  6.5013, Val:  6.5017
After 3 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 4 iterations is: Train:  6.3007, Val:  6.3013
After 4 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 5 iterations is: Train:  6.1081, Val:  6.1088
After 5 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 6 iterations is: Train:  5.9219, Val:  5.9229
After 6 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 7 iterations is: Train:  5.7409, Val:  5.7421
After 7 iterations: Val Acc: 82.08697675320394; Train Acc: 82.29373405160193
The cost after 8 iterations is: Tr